In [6]:
class Quantity:
    def __init__(self, storage_name):
        self.storage_name = storage_name

    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self.storage_name]

In [7]:
class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [15]:
# simplify Quantity by automatic naming
class Quantity:
    def __set_name__(self, owner, name):
        self.storage_name = name

    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)

In [19]:
# using the above descriptor instance Quantity
# descriptors seperated into separate files: reusability
class LineItem:
    weight = Quantity() # similar to Django model fields are descriptors
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [20]:
import abc

class Validated(abc.ABC):
    def __set_name__(self, owner, name):
        self.storage_name = name

    def __set__(self, instance, value):
        value = self.validate(self.storage_name, value)
        instance.__dict__[self.storage_name] = value

    @abc.abstractmethod
    def validate(self, name, value):
        """return validated value or raise ValueError"""

In [21]:
class Quantity(Validated):
    """a number greater than zero"""
    def validate(self, name, value):
        if value <= 0:
            raise ValueError(f'{name} must be > 0')
        return value

class NonBlank(Validated):
    """a string with at least one non-space character"""
    def validate(self, name, value):
        value = value.strip()
        if not value:
            raise ValueError(f'{name} cannot be blank')
        return value

In [22]:
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [24]:
# aux functions for display 
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return f'<class {obj.__name__}>'
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return f'<{cls_name(obj)} object>'

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print(f'-> {cls_name(args[0])}.__{name}__({pseudo_args})')

In [26]:
"""
Overriding descriptors are also called “enforced descrip‐
tors.” Synonyms for nonoverriding descriptors include “nondata
descriptors” or “shadowable descriptors.”
"""
class Overriding:
    """data descriptor or enforced descriptor"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

    def __set__(self, instance, value):
        print_args('set', self, instance, value)

class OverridingNoGet:
    """an overriding descriptor without `__get__`"""
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

class NonOverriding:
    """non-data or shadowable desciptor"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):
        print(f'-> Managed.span({display(self)})')

In [27]:
obj = Managed()
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [28]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [29]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [30]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [32]:
obj.__dict__['over'] = 8
vars(obj)

{'over': 8}

In [33]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [34]:
obj.over_no_get

In [35]:
Managed.over_no_get

In [36]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [37]:
obj.over_no_get

In [38]:
obj.__dict__['over_no_get'] = 9
obj.over_no_get

9

In [39]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [40]:
obj.over_no_get

9

In [41]:
obj = Managed()
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [42]:
obj.non_over = 7
obj.non_over

7

In [43]:
Managed.non_over

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [44]:
del obj.non_over

In [45]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [48]:
"""
To control the setting of attributes in a class, you have to
attach descriptors to the class of the class—in, the metaclass.
"""
obj = Managed()
Managed.over = 1
Managed.over_no_get = 2
Managed.non_over = 3
obj.over, obj.over_no_get, obj.non_over

(1, 2, 3)

In [49]:
# a method is a nonoverriding descriptor
obj = Managed()
obj.spam

<bound method Managed.spam of <__main__.Managed object at 0x103b8dd20>>

In [50]:
Managed.spam

<function __main__.Managed.spam(self)>

In [51]:
obj.spam = 7
obj.spam

7

In [52]:
import collections

class Text(collections.UserString):
    def __repr__(self):
        return 'Text({!r})'.format(self.data)

    def reverse(self):
        return self[::-1]

In [53]:
word = Text('pizza')
word

Text('pizza')

In [54]:
word.reverse()

Text('azzip')

In [55]:
Text.reverse(Text('backwards'))

Text('sdrawkcab')

In [56]:
type(Text.reverse), type(word.reverse)

(function, method)

In [57]:
list(map(Text.reverse, ['repaid', (10,20,30), Text('strong')]))

['diaper', (30, 20, 10), Text('gnorts')]

In [58]:
Text.reverse.__get__(word)

<bound method Text.reverse of Text('pizza')>

In [59]:
Text.reverse.__get__(None, Text)

<function __main__.Text.reverse(self)>

In [60]:
word.reverse

<bound method Text.reverse of Text('pizza')>

In [61]:
word.reverse.__self__

Text('pizza')

In [62]:
word.reverse.__func__ is Text.reverse

True